In [1]:
import os

os.environ["OMP_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"


import numpy as np

from QC_DFT.NeuralNetwork import QCDFTModel
from QC_DFT.Circuit import QCDFT_Circuit, NN_QCDFT_Circuit
from QC_DFT.Utils import Gates
import pennylane as qml

from tqdm import tqdm

import random
import copy
import time

In [ ]:
def generate_random_gates(n_qubits, gate_count):
    gates = ["X", "Y", "Z", "H", "CX", "CX", "CX", "CX", "CX", "CX", "CX", "CX", "RX", "RY", "RZ"]
    qubits = list(range(n_qubits))

    res = []

    for i in range(gate_count):
        gate = random.choice(gates)
        qubit = random.choice(qubits)
        param = None
        if gate == "CX":
            qubit2 = None
            while qubit2 == None or qubit2 == qubit:
                qubit2 = random.choice(qubits)
            qubit = (qubit, qubit2)
        if gate == "RX" or gate == "RY" or gate == "RZ":
            param = random.uniform(0, 2 * np.pi)
        res += [(gate, qubit, param)]
    
    return res

# gates = (gate(string), qubit, params)
def apply_gate_circuit(circuit, gate):
    gate, qubit, params = gate
    if gate == "X":
        circuit.x(qubit)
    if gate == "Y":
        circuit.y(qubit)
    if gate == "Z":
        circuit.z(qubit)
    if gate == "H":
        circuit.h(qubit)
    if gate == "I":
        circuit.i(qubit)
    if gate == "RX":
        circuit.rx(params, qubit)
    if gate == "RY":
        circuit.ry(params, qubit)
    if gate == "RZ":
        circuit.rz(params, qubit)
    if gate == "CX":
        circuit.cx(*qubit)

    return circuit

def apply_gate_exact(state, gate):
    n_qubits = int(np.log2(state.shape[0]))
    gate, qubit, params = gate

    unitary = None
    if gate == "X":
        unitary = Gates.X()
    if gate == "Y":
        unitary = Gates.Y()
    if gate == "Z":
        unitary = Gates.Z()
    if gate == "H":
        unitary = Gates.H()
    if gate == "I":
        unitary = Gates.I()
    if gate == "RX":
        unitary = Gates.RX(params)
    if gate == "RY":
        unitary = Gates.RY(params)
    if gate == "RZ":
        unitary = Gates.RZ(params)

    if unitary is not None:
        for _ in range(qubit):
            unitary = np.kron(Gates.I(), unitary)
        for _ in range(qubit + 1, n_qubits):
            unitary = np.kron(unitary, Gates.I())

    if gate == "CX":
        unitary = np.zeros((2 ** n_qubits, 2 ** n_qubits), dtype=complex)
        for i in range(2 ** n_qubits):
            if i & (2 ** qubit[0]) != 0:
                unitary[i][i ^ (2 ** qubit[1])] = 1
            else:
                unitary[i][i] = 1

    return unitary @ state @ np.transpose(np.conjugate(unitary))

def get_sqp_exact(state):
    n_qubits = int(np.log2(state.shape[0]))

    sqps = [0.0 for _ in range(n_qubits)]

    for i in range(2 ** n_qubits):
        for j in range(n_qubits):
            if i & (2 ** j) != 0:
                sqps[j] += state[i][i] 

    return np.array(sqps, dtype=float)


def get_fidelity(state, rdms):
    n_qubits = int(np.log2(state.shape[0]))
    trace_list = list(range(n_qubits))
    fidelity = []
    for i in range(n_qubits):
        trace_list_ = copy.deepcopy(trace_list)
        trace_list_.remove(i)
        fidelity += [qml.math.fidelity(qml.math.partial_trace(state, trace_list_), rdms[i])]
    return np.array(fidelity)

In [3]:
nn_models = QCDFTModel.load("Models/default")

In [ ]:
print(nn_models.control_nn)
print(nn_models.target_nn)

In [ ]:
max_n_qubits = 12
gate_count = 10
iteration_count = 10

In [ ]:
exact_time_mean = []
bernardi_time_mean = []
proposed_time_mean = []

for n_qubits in tqdm(range(2, max_n_qubits)):
    exact_time_mean_inner = []
    bernardi_time_mean_inner = []
    proposed_time_mean_inner = []

    for iteration in tqdm(range(iteration_count), leave=False):
        exact_state = np.zeros((2 ** n_qubits, 2 ** n_qubits), dtype=complex)
        exact_state[0][0] = 1
        bernardi_circuit = QCDFT_Circuit(n_qubits)
        proposed_circuit = NN_QCDFT_Circuit(n_qubits, nn_models)

        gates = generate_random_gates(n_qubits, gate_count)
        for gate in gates:
            bernardi_circuit = apply_gate_circuit(bernardi_circuit, gate)
            proposed_circuit = apply_gate_circuit(proposed_circuit, gate)

        time_exact = time.time()
        for operator_iterator in range(gate_count):
            exact_state = apply_gate_exact(exact_state, gates[operator_iterator])
        time_exact = time.time() - time_exact

        time_bernardi = time.time()
        for operator_iterator in range(gate_count):
            bernardi_state = bernardi_circuit.evolve(1)
        time_bernardi = time.time() - time_bernardi

        time_proposed = time.time()
        for operator_iterator in range(gate_count):
            proposed_state = proposed_circuit.evolve(1)
        time_proposed = time.time() - time_proposed

        exact_time_mean_inner += [time_exact]
        bernardi_time_mean_inner += [time_bernardi]
        proposed_time_mean_inner += [time_proposed]

    exact_time_mean += [exact_time_mean_inner]
    bernardi_time_mean += [bernardi_time_mean_inner]
    proposed_time_mean += [proposed_time_mean_inner]

exact_time_mean = np.mean(np.array(exact_time_mean, dtype=float), axis=1)
bernardi_time_mean = np.mean(np.array(bernardi_time_mean, dtype=float), axis=1)
proposed_time_mean = np.mean(np.array(proposed_time_mean, dtype=float), axis=1)


In [7]:
folder = "Results"
suffix = ""

np.save(f"{folder}/exact_time_mean{suffix}", exact_time_mean)
np.save(f"{folder}/bernardi_time_mean{suffix}", bernardi_time_mean)
np.save(f"{folder}/proposed_time_mean{suffix}", proposed_time_mean)